### Importar arquivos e bibliotecas:

In [1]:
#esperar navegador carregar:
import time

#determinar data:
from datetime import date

#importar e exportar planilhas:
import pandas as pd
from pathlib import Path

#anexar arquivos ao e-mail e enviar:
import smtplib
import email.message
import mimetypes
from email.message import EmailMessage

#localizar pastas:
caminho = Path.cwd() #localizar pasta Projeto AutomacaoWeb
pasta_planilhas_ofertas = Path(caminho/Path(r'Tabelas de Preços')) #localizar pasta para salvar planilhas

### Criar navegador:

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

#automaticamente atualizar ChromeDriver para versão mais recente no momento em que o programa rodar:
servico = Service(ChromeDriverManager().install())

#definir variável para o navegador (driver):
navegador = webdriver.Chrome(service = servico)

### Abrir dataframe de buscas a serem feitas:

In [3]:
buscas_df = pd.read_excel('buscas.xlsx')
display(buscas_df)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


### Pegar ofertas no Google Shopping de apenas um produto (iPhone):

In [4]:
links = [] #lista de links para as ofertas
lista_precos = [] #lista dos preços de cada oferta
lista_linhas_com_produto = [] #lista usada pelo dataframe de ofertas para indicar qual produto foi buscado em cada linha
lista_nomes_ofertas = [] #lista com nome de cada oferta

#exemplo de variáveis usadas para busca de um produto:
nome_produto = 'iphone 12 64gb' 
termos_banidos = 'mini watch'
termos_banidos = termos_banidos.split()
preco_min = 3000
preco_max = 3500

#abrir site do Google Shopping:
navegador.get('https://www.google.com/search?q=redirecionando...&sa=X&tbm=shop&sxsrf=ALiCzsZca8FJL977pr8MQn2epVnpew8ZUw%3A1664547658490&ei=Svs2Y5SdHbnI1sQP6NSD6Ag&ved=0ahUKEwiU6pSf27z6AhU5pJUCHWjqAI0Q4dUDCAY&uact=5&oq=redirecionando...&gs_lcp=Cgtwcm9kdWN0cy1jYxADSgQIQRgAUABY6wJg4gdoAHAAeACAAc8BiAHBBJIBBTAuMi4xmAEAoAEBwAEB&sclient=products-cc')
    
#pesquisar pelo produto:
pesquisa_input = navegador.find_element(By.XPATH, '//*[@id="tsf"]/div[1]/div[1]/div[2]/div/div[2]/input')
pesquisa_input.clear()
pesquisa_input.send_keys(nome_produto)
pesquisa_input.send_keys(Keys.ENTER)

#filtrar preço do produto:

preco_min_input = navegador.find_element(By.CSS_SELECTOR, '[aria-label="Preço mínimo"]')
preco_min_input.send_keys(preco_min)

preco_max_input = navegador.find_element(By.CSS_SELECTOR, '[aria-label="Preço máximo"]')
preco_max_input.send_keys(preco_max)

#buscar por produto com o filtro de preços aplicado:
preco_max_input.send_keys(Keys.ENTER)

time.sleep(3)

links_a_descartar = [] #lista usada para filtrar lista de links, removendo links com termos banidos

#preencher lista de links, títulos das ofertas e preços não formatados:

anchor_tag_com_links = navegador.find_elements(By.CSS_SELECTOR, 'a.xCpuod') 

for elemento in anchor_tag_com_links:
    links.append(elemento.get_attribute('href'))

links.remove(links[0]) #remover primeira oferta com formatação diferente das demais

titulos_ofertas = navegador.find_elements(By.CSS_SELECTOR, 'h4.Xjkr3b')
titulos_ofertas.remove(titulos_ofertas[0]) #remover primeira oferta com formatação diferente das demais

precos_nao_formatados = navegador.find_elements(By.CSS_SELECTOR, 'span.a8Pemb.OFFNJ')
precos_nao_formatados.remove(precos_nao_formatados[0]) #remover primeira oferta com formatação diferente das demais

#percorrer todas as ofertas a partir da lista de títulos:
for i, titulo in enumerate(titulos_ofertas):

    link_mal = False #link inicialmente não precisa ser descartado
    titulo = titulo.text.lower() #transformar elemento do título em texto e transformar em lowercase para tratamento

    for termo in termos_banidos:
        if termo in titulo:
            link_mal = True #link precisa ser descartado por conter um termo banido
            break

    if link_mal is True:
        links_a_descartar.append(links[i]) #acrescentar link mal a lista de links a descartar

    else:
        titulo = titulo.upper() #deixar o título em uppercase para apresentar no dataframe
        lista_nomes_ofertas.append(titulo)
        
        preco = precos_nao_formatados[i]
        
        #formatar preço:
        preco = preco.text
        preco = preco[3:11]
        preco = preco.replace('.', '')
        preco = preco.replace(',', '.')
        preco = float(preco)

        lista_precos.append(preco)
        lista_linhas_com_produto.append(nome_produto)

for link_descartavel in links_a_descartar:
    links.remove(link_descartavel) #remover links descartáveis da lista de links

lista_linhas_com_produto = [nome.upper() for nome in lista_linhas_com_produto]

#criar dataframe de ofertas:

dicionario_ofertas_google_shop = {'Busca': lista_linhas_com_produto, 'Nome da Oferta': lista_nomes_ofertas,
                                  'Preço': lista_precos, 'Link': links}

ofertas_produto_df_google_shop = pd.DataFrame.from_dict(dicionario_ofertas_google_shop)

display(ofertas_produto_df_google_shop)

,Busca,Nome da Oferta,Preço,Link
0,IPHONE 12 64GB,IPHONE APPLE 11 64 GB PRETO,3229.05,https://www.google.com/shopping/product/399973...
1,IPHONE 12 64GB,"APPLE IPHONE 12, 64 GB, VERDE - TOTALMENTE DES...",3175.00,https://www.google.com/shopping/product/166124...
2,IPHONE 12 64GB,"APPLE IPHONE 12, 64 GB, (PRODUTO) VERMELHO - T...",3089.00,https://www.google.com/shopping/product/877726...
3,IPHONE 12 64GB,"VITRINE: IPHONE 11 APPLE 64GB PRETO TELA 6,1 C...",3099.00,https://www.google.com/shopping/product/311403...
4,IPHONE 12 64GB,"VITRINE: IPHONE 11 APPLE 64GB ROXO TELA 6,1 CÂ...",3099.00,https://www.google.com/shopping/product/164899...
5,IPHONE 12 64GB,IPHONE APPLE XR 65GB CÂMERA 12 MP TELA 6.1 DUA...,3389.00,https://www.google.com/url?url=https://www.car...
6,IPHONE 12 64GB,DE SEGUNDA MÃO 100% ORIGINAL DE SEGUNDA MÃO 90...,3182.93,https://www.google.com/url?url=https://s.click...
7,IPHONE 12 64GB,IPHONE 11 PRO - 64 GB - 12X SEM JUROS NO CARTÃO !,3490.00,https://www.google.com/url?url=https://pr.olx....
8,IPHONE 12 64GB,SMARTPHONE APPLE IPHONE 11 64GB BRANCO,3096.00,https://www.google.com/url?url=https://www.zoo...
9,IPHONE 12 64GB,SMARTPHONE APPLE IPHONE 11 BRANCO 64 GB,3188.97,https://www.google.com/shopping/product/129074...


### Pegar oferta de vários produtos no Google Shopping:

In [5]:
links = [] #lista de links para as ofertas
lista_precos = [] #lista dos preços de cada oferta
lista_linhas_com_produto = [] #lista usada pelo dataframe de ofertas para indicar qual produto foi buscado em cada linha
lista_nomes_ofertas = [] #lista com nome de cada oferta

for produto in buscas_df.index:
    nome_produto = buscas_df['Nome'][produto]
    termos_banidos = buscas_df['Termos banidos'][produto]
    termos_banidos = termos_banidos.split()
    preco_min = int(buscas_df['Preço mínimo'][produto])
    preco_max = int(buscas_df['Preço máximo'][produto])

    #abrir site do Google Shopping:
    navegador.get('https://www.google.com/search?q=redirecionando...&sa=X&tbm=shop&sxsrf=ALiCzsZca8FJL977pr8MQn2epVnpew8ZUw%3A1664547658490&ei=Svs2Y5SdHbnI1sQP6NSD6Ag&ved=0ahUKEwiU6pSf27z6AhU5pJUCHWjqAI0Q4dUDCAY&uact=5&oq=redirecionando...&gs_lcp=Cgtwcm9kdWN0cy1jYxADSgQIQRgAUABY6wJg4gdoAHAAeACAAc8BiAHBBJIBBTAuMi4xmAEAoAEBwAEB&sclient=products-cc')

    #pesquisar pelo produto:
    pesquisa_input = navegador.find_element(By.XPATH, '//*[@id="tsf"]/div[1]/div[1]/div[2]/div/div[2]/input')
    pesquisa_input.clear()
    pesquisa_input.send_keys(nome_produto)
    pesquisa_input.send_keys(Keys.ENTER)

    #filtrar preço do produto:

    preco_min_input = navegador.find_element(By.CSS_SELECTOR, '[aria-label="Preço mínimo"]')
    preco_min_input.send_keys(preco_min)

    preco_max_input = navegador.find_element(By.CSS_SELECTOR, '[aria-label="Preço máximo"]')
    preco_max_input.send_keys(preco_max)

    #buscar por produto com o filtro de preços aplicado:
    preco_max_input.send_keys(Keys.ENTER)

    links_a_descartar = [] #lista usada para filtrar lista de links, removendo links com termos banidos

    #preencher lista de links, títulos das ofertas e preços não formatados:
    
    anchor_tag_com_links = navegador.find_elements(By.CSS_SELECTOR, 'a.xCpuod') 

    for elemento in anchor_tag_com_links:
        links.append(elemento.get_attribute('href'))
        
    links.remove(links[0]) #remover primeira oferta com formatação diferente das demais

    titulos_ofertas = navegador.find_elements(By.CSS_SELECTOR, 'h4.Xjkr3b')
    titulos_ofertas.remove(titulos_ofertas[0]) #remover primeira oferta com formatação diferente das demais
    precos_nao_formatados = navegador.find_elements(By.CSS_SELECTOR, 'span.a8Pemb.OFFNJ')
    precos_nao_formatados.remove(precos_nao_formatados[0]) #remover primeira oferta com formatação diferente das demais

    #percorrer todas as ofertas a partir da lista de títulos:
    for i, titulo in enumerate(titulos_ofertas):
        
        link_mal = False #link inicialmente não precisa ser descartado
        titulo = titulo.text.lower() #transformar elemento do título em texto e transformar em lowercase para tratamento

        for termo in termos_banidos:
            if termo in titulo:
                link_mal = True #link precisa ser descartado por conter um termo banido
                break

        if link_mal is True:
            links_a_descartar.append(links[i]) #acrescentar link mal a lista de links a descartar

        else:
            titulo = titulo.upper()
            lista_nomes_ofertas.append(titulo) #deixar o título em uppercase para apresentar no dataframe
        
            preco = precos_nao_formatados[i]

            #formatar preço:
            preco = preco.text
            preco = preco[3:11]
            preco = preco.replace('.', '')
            preco = preco.replace(',', '.')
            preco = float(preco)

            lista_precos.append(preco)
            lista_linhas_com_produto.append(nome_produto)

    for link_descartavel in links_a_descartar:
        links.remove(link_descartavel) #remover links descartáveis da lista de links

lista_linhas_com_produto = [nome.upper() for nome in lista_linhas_com_produto]

#criar dataframe de ofertas:

dicionario_ofertas_google_shop = {'Busca': lista_linhas_com_produto, 'Nome da Oferta': lista_nomes_ofertas,
                                  'Preço': lista_precos, 'Link': links}

ofertas_produto_df_google_shop = pd.DataFrame.from_dict(dicionario_ofertas_google_shop)

display(ofertas_produto_df_google_shop)

,Busca,Nome da Oferta,Preço,Link
0,IPHONE 12 64GB,IPHONE APPLE 11 64 GB PRETO,3229.05,https://www.google.com/shopping/product/166124...
1,IPHONE 12 64GB,"APPLE IPHONE 12, 64 GB, VERDE - TOTALMENTE DES...",3175.00,https://www.google.com/shopping/product/877726...
2,IPHONE 12 64GB,"APPLE IPHONE 12, 64 GB, (PRODUTO) VERMELHO - T...",3089.00,https://www.google.com/shopping/product/311403...
3,IPHONE 12 64GB,"VITRINE: IPHONE 11 APPLE 64GB PRETO TELA 6,1 C...",3099.00,https://www.google.com/shopping/product/164899...
4,IPHONE 12 64GB,"VITRINE: IPHONE 11 APPLE 64GB ROXO TELA 6,1 CÂ...",3099.00,https://www.google.com/url?url=https://www.car...
...,...,...,...,...
108,RTX 3060,PLACA DE VÍDEO MSI GEFORCE RTX 3060 TI VENTUS ...,4029.00,https://www.google.com/shopping/product/118081...
109,RTX 3060,NOVO LANÇADO GAMING PC AMD RYZEN 9 5900HX RTX3...,4236.32,https://www.google.com/url?url=https://s.click...
110,RTX 3060,TOPTON GAMING COMPUTADOR DESKTOP AMD RYZEN 9 5...,4115.26,https://www.google.com/url?url=https://s.click...
111,RTX 3060,NOVO LA-K452P I7-11800H PARA DELL ALIENWARE M1...,4189.83,https://www.google.com/url?url=https://s.click...


### Pegar ofertas no Buscapé de apenas um produto (iPhone):

In [6]:
links = [] #lista de links para as ofertas
lista_precos = [] #lista dos preços de cada oferta
lista_linhas_com_produto = [] #lista usada pelo dataframe de ofertas para indicar qual produto foi buscado em cada linha
lista_nomes_ofertas = [] #lista com nome de cada oferta

#exemplo de variáveis usadas para busca de um produto:
nome_produto = 'iphone 12 64gb' 
termos_banidos = 'mini watch'
termos_banidos = termos_banidos.split()
preco_min = 3000
preco_max = 3500

#abrir site do Buscapé:
navegador.get('https://www.buscape.com.br/?og=19220&og=19220&gclid=EAIaIQobChMI7ffW79O3-gIVYuhcCh3OigqeEAAYASAAEgJOg_D_BwE')

#concordar com política de privacidade, se necessário:

botao_privacidade = navegador.find_elements(By.CSS_SELECTOR, '#__next > div.PrivacyPolicy_floating__bj1Ov > div > div > div.PrivacyPolicy_buttonCell__xZTNK.col-lg-3 > button')

if len(botao_privacidade) != 0:
    botao_privacidade[0].click()

#pesquisar pelo produto:
pesquisa_input = navegador.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input')
pesquisa_input.send_keys(nome_produto)
pesquisa_input.send_keys(Keys.ENTER)

#esperar página carregar:
while len(navegador.find_elements(By.XPATH, '//*[@id="__next"]/div[2]/div/div[1]/div[2]/aside/div[2]/div/div[2]/div[1]/div/input')) == 0:
    time.sleep(1)
time.sleep(1)

#filtrar preço do produto:

preco_min_input = navegador.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/div[1]/div[2]/aside/div[2]/div/div[2]/div[1]/div/input')
preco_min_input.send_keys(10 * Keys.BACKSPACE)
preco_min_input.send_keys(preco_min)
preco_min_input.send_keys('00')

preco_max_input = navegador.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/div[1]/div[2]/aside/div[2]/div/div[2]/div[2]/div/input')
preco_max_input.send_keys(10 * Keys.BACKSPACE)
preco_max_input.send_keys(preco_max)
preco_max_input.send_keys('00')

#buscar por produto com o filtro de preços aplicado:
botao_pesquisar = navegador.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/div[1]/div[2]/aside/div[2]/div/div[2]/button')
botao_pesquisar.click()

time.sleep(3)

links_a_descartar = [] #lista usada para filtrar lista de links, removendo links com termos banidos

#preencher lista de links, títulos das ofertas e preços não formatados:

anchor_tag_com_links = navegador.find_elements(By.CSS_SELECTOR, 'a.SearchCard_ProductCard_Inner__7JhKb')

for elemento in anchor_tag_com_links:
    links.append(elemento.get_attribute('href'))

titulos_ofertas = navegador.find_elements(By.CSS_SELECTOR, '[data-testid="product-card::name"]')
precos_nao_formatados = navegador.find_elements(By.CSS_SELECTOR, '[data-testid="product-card::price"]')

#percorrer todas as ofertas a partir da lista de títulos:
for i, titulo in enumerate(titulos_ofertas):

    link_mal = False #link inicialmente não precisa ser descartado
    titulo = titulo.text.lower() #transformar elemento do título em texto e transformar em lowercase para tratamento

    for termo in termos_banidos:
        if termo in titulo:
            link_mal = True #link precisa ser descartado por conter um termo banido
            break

    if link_mal is True:
        links_a_descartar.append(links[i]) #acrescentar link mal a lista de links a descartar

    else:
        titulo = titulo.upper()
        lista_nomes_ofertas.append(titulo) #deixar o título em uppercase para apresentar no dataframe
        
        preco = precos_nao_formatados[i]

        #formatar preço:
        preco = preco.text
        preco = preco[3:11]
        preco = preco.replace('.', '')
        preco = preco.replace(',', '.')
        preco = float(preco)

        lista_precos.append(preco)
        lista_linhas_com_produto.append(nome_produto)

for link_descartavel in links_a_descartar:
    links.remove(link_descartavel) #remover links descartáveis da lista de links
    
lista_linhas_com_produto = [nome.upper() for nome in lista_linhas_com_produto]

#criar dataframe de ofertas:

dicionario_ofertas_buscape = {'Busca': lista_linhas_com_produto, 'Nome da Oferta': lista_nomes_ofertas,
                                  'Preço': lista_precos, 'Link': links}


ofertas_produto_df_buscape = pd.DataFrame.from_dict(dicionario_ofertas_buscape)

display(ofertas_produto_df_buscape)

,Busca,Nome da Oferta,Preço,Link
0,IPHONE 12 64GB,SMARTPHONE APPLE IPHONE 11 64GB CÂMERA DUPLA,3096.0,https://www.buscape.com.br/celular/smartphone-...
1,IPHONE 12 64GB,SMARTPHONE APPLE IPHONE SE 3 64GB 12.0 MP,3099.0,https://www.buscape.com.br/celular/smartphone-...
2,IPHONE 12 64GB,SMARTPHONE APPLE IPHONE SE 2 VERMELHO 64GB 12....,3389.5,https://www.buscape.com.br/celular/smartphone-...
3,IPHONE 12 64GB,SMARTPHONE APPLE IPHONE 11 VERMELHO 64GB CÂMER...,3328.2,https://www.buscape.com.br/celular/smartphone-...
4,IPHONE 12 64GB,SMARTPHONE APPLE IPHONE 11 PRO USADO 64GB CÂME...,3419.1,https://www.buscape.com.br/celular/smartphone-...


### Pegar oferta de vários produtos no Buscapé:

In [7]:
links = [] #lista de links para as ofertas
lista_precos = [] #lista dos preços de cada oferta
lista_linhas_com_produto = [] #lista usada pelo dataframe de ofertas para indicar qual produto foi buscado em cada linha
lista_nomes_ofertas = [] #lista com nome de cada oferta

for produto in buscas_df.index:
    nome_produto = buscas_df['Nome'][produto]
    termos_banidos = buscas_df['Termos banidos'][produto]
    termos_banidos = termos_banidos.split()
    preco_min = int(buscas_df['Preço mínimo'][produto])
    preco_max = int(buscas_df['Preço máximo'][produto])
    
    #abrir site do Buscapé:
    navegador.get('https://www.buscape.com.br/?og=19220&og=19220&gclid=EAIaIQobChMI7ffW79O3-gIVYuhcCh3OigqeEAAYASAAEgJOg_D_BwE')

    #concordar com política de privacidade, se necessário:

    botao_privacidade = navegador.find_elements(By.CSS_SELECTOR, '#__next > div.PrivacyPolicy_floating__bj1Ov > div > div > div.PrivacyPolicy_buttonCell__xZTNK.col-lg-3 > button')

    if len(botao_privacidade) != 0:
        botao_privacidade[0].click()

    #pesquisar pelo produto:
    pesquisa_input = navegador.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input')
    pesquisa_input.send_keys(nome_produto)
    pesquisa_input.send_keys(Keys.ENTER)

    #esperar página carregar:
    while len(navegador.find_elements(By.XPATH, '//*[@id="__next"]/div[2]/div/div[1]/div[2]/aside/div[2]/div/div[2]/div[1]/div/input')) == 0:
        time.sleep(1)
    time.sleep(1)

    #filtrar preço do produto:

    preco_min_input = navegador.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/div[1]/div[2]/aside/div[2]/div/div[2]/div[1]/div/input')
    preco_min_input.send_keys(10 * Keys.BACKSPACE)
    preco_min_input.send_keys(preco_min)
    preco_min_input.send_keys('00')

    preco_max_input = navegador.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/div[1]/div[2]/aside/div[2]/div/div[2]/div[2]/div/input')
    preco_max_input.send_keys(10 * Keys.BACKSPACE)
    preco_max_input.send_keys(preco_max)
    preco_max_input.send_keys('00')

    #buscar por produto com o filtro de preços aplicado:
    botao_pesquisar = navegador.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/div[1]/div[2]/aside/div[2]/div/div[2]/button')
    botao_pesquisar.click()

    time.sleep(3)

    links_a_descartar = [] #lista usada para filtrar lista de links, removendo links com termos banidos

    #preencher lista de links, títulos das ofertas e preços não formatados:

    anchor_tag_com_links = navegador.find_elements(By.CSS_SELECTOR, 'a.SearchCard_ProductCard_Inner__7JhKb')

    for elemento in anchor_tag_com_links:
        links.append(elemento.get_attribute('href'))

    titulos_ofertas = navegador.find_elements(By.CSS_SELECTOR, '[data-testid="product-card::name"]')
    precos_nao_formatados = navegador.find_elements(By.CSS_SELECTOR, '[data-testid="product-card::price"]')

    #percorrer todas as ofertas a partir da lista de títulos:
    for i, titulo in enumerate(titulos_ofertas):

        link_mal = False #link inicialmente não precisa ser descartado
        titulo = titulo.text.lower() #transformar elemento do título em texto e transformar em lowercase para tratamento

        for termo in termos_banidos:
            if termo in titulo:
                link_mal = True #link precisa ser descartado por conter um termo banido
                break

        if link_mal is True:
            links_a_descartar.append(links[i]) #acrescentar link mal a lista de links a descartar

        else:
            titulo = titulo.upper()
            lista_nomes_ofertas.append(titulo) #deixar o título em uppercase para apresentar no dataframe

            preco = precos_nao_formatados[i]

            #formatar preço:
            preco = preco.text
            preco = preco[3:11]
            preco = preco.replace('.', '')
            preco = preco.replace(',', '.')
            preco = float(preco)

            lista_precos.append(preco)
            lista_linhas_com_produto.append(nome_produto)

    for link_descartavel in links_a_descartar:
        links.remove(link_descartavel) #remover links descartáveis da lista de links
        
lista_linhas_com_produto = [nome.upper() for nome in lista_linhas_com_produto]

#criar dataframe de ofertas:

dicionario_ofertas_buscape = {'Busca': lista_linhas_com_produto, 'Nome da Oferta': lista_nomes_ofertas,
                                  'Preço': lista_precos, 'Link': links}


ofertas_produto_df_buscape = pd.DataFrame.from_dict(dicionario_ofertas_buscape)

display(ofertas_produto_df_buscape)

,Busca,Nome da Oferta,Preço,Link
0,IPHONE 12 64GB,SMARTPHONE APPLE IPHONE 11 64GB CÂMERA DUPLA,3096.00,https://www.buscape.com.br/celular/smartphone-...
1,IPHONE 12 64GB,SMARTPHONE APPLE IPHONE SE 3 64GB 12.0 MP,3099.00,https://www.buscape.com.br/celular/smartphone-...
2,IPHONE 12 64GB,SMARTPHONE APPLE IPHONE SE 2 VERMELHO 64GB 12....,3389.50,https://www.buscape.com.br/celular/smartphone-...
3,IPHONE 12 64GB,SMARTPHONE APPLE IPHONE 11 VERMELHO 64GB CÂMER...,3328.20,https://www.buscape.com.br/placa-de-video/plac...
4,IPHONE 12 64GB,SMARTPHONE APPLE IPHONE 11 PRO USADO 64GB CÂME...,3419.10,https://www.buscape.com.br/lead?oid=533936627&...
5,RTX 3060,PLACA DE VIDEO NVIDIA GEFORCE RTX 3060 TI 8 GB...,4227.26,https://www.buscape.com.br/lead?oid=860957132&...
6,RTX 3060,PLACA DE VIDEO NVIDIA GEFORCE RTX 3060 12 GB G...,4028.26,https://www.buscape.com.br/lead?oid=822005184&...
7,RTX 3060,PLACA DE VÍDEO NVIDIA GEFORCE RTX 3060 TI FOUN...,4500.00,https://www.buscape.com.br/lead?oid=724728410&...
8,RTX 3060,PLACA DE VÍDEO ASUS RTX 3060 DUAL LHR V2 12GB ...,4399.50,https://www.buscape.com.br/lead?oid=582225866&...
9,RTX 3060,GIGABYTE PLACA DE VÍDEO GEFORCE RTX 3060 EAGLE...,4103.58,https://www.buscape.com.br/lead?oid=546314735&...


In [8]:
navegador.quit() #fechar o navegador

### Salvar planilhas em uma pasta de backup:

In [9]:
#registrar data de pesquisa:
data = date.today()
data_formatada = data.strftime('%d_%m_') #formatação usada no nome dos arquivos

ofertas_produto_df_google_shop.to_excel(
    str(pasta_planilhas_ofertas) + '/{}Tabela de Preços Google Shopping.xlsx'.format(data_formatada), index=False)
str_tabela_google_shop = str(pasta_planilhas_ofertas) + '/{}Tabela de Preços Google Shopping.xlsx'.format(data_formatada)

ofertas_produto_df_buscape.to_excel(
    str(pasta_planilhas_ofertas) + '/{}Tabela de Preços Buscapé.xlsx'.format(data_formatada), index=False)
str_tabela_buscape = str(pasta_planilhas_ofertas) + '/{}Tabela de Preços Buscapé.xlsx'.format(data_formatada)

#registrar quais produtos foram pesquisados:
lista_produtos_pesquisados = buscas_df['Nome'].tolist()

### Enviar e-mail para área de compras:

In [10]:
def enviar_email(lista_produtos_pesquisados, data, arquivo_google_shop, arquivo_buscape):
    
    '''Envia e-mail contendo ofertas dos produtos para a área de compras.
    
    Parameters:
    lista_produtos_pesquisados(list): lista dos produtos pesquisados;
    data(datetime.date): data de hoje;
    arquivo_google_shop(str): nome do arquivo de ofertas do Google Shop a ser anexado;
    arquivo_buscape(str): nome do arquivo de ofertas do Buscapé a ser anexado.
    
    Returns:
    None (apenas faz print do status de envio, confirmando que o e-mail foi enviado).
    '''
    
    tupla_produtos = tuple(lista_produtos_pesquisados) #apresentar os produtos pesquisados entre parênteses (mais formal assim)
    
    corpo_email = """
    <p>Bom dia,</p>
    <p>Segue em anexo as tabelas dos preços dos produtos requisitados {} contendo preço, nome do produto e link para a oferta. Qualquer dúvida, estou à disposição.</p>
    <p></p>
    <p>Att.,</p>
    <p>Jovi Prata</p>
    """.format(tupla_produtos)

    data_formatada = data.strftime('%d_%m_') #formatação usada no nome dos arquivos
    data_reformatada = data.strftime('%d/%m') #formatação diferente usada no assunto do e-mail
    
    msg = EmailMessage()
    msg['Subject'] = "Tabelas de Preços dos Produtos Requisitados Dia {}".format(data_reformatada)
    msg['From'] = 'jovitrash@gmail.com' #e-mail de exemplo (usado apenas para testes)
    msg['To'] = 'jovitrash+areadecompras@gmail.com' #e-mail de exemplo para área de compras (usado apenas para testes)
    password = 'emqbzkitmdfdtzxq' 
    msg.add_header('Content-Type', 'text/html')
    msg.set_content(corpo_email, subtype='html')
    
    
    mime_type, encoding = mimetypes.guess_type(arquivo_google_shop)
    with open(arquivo_google_shop, 'rb') as fp:
        dados = fp.read()

    msg.add_attachment(dados, maintype=mime_type.split("/")[0], subtype=mime_type.split("/")[1],
                       filename="{}Tabela de Preços Google Shopping.xlsx".format(data_formatada))
    
    
    mime_type, encoding = mimetypes.guess_type(arquivo_buscape)
    with open(arquivo_buscape, 'rb') as fp:
        dados = fp.read()

    msg.add_attachment(dados, maintype=mime_type.split("/")[0], subtype=mime_type.split("/")[1],
                       filename=r"{}Tabela de Preços Buscapé.xlsx".format(data_formatada))

    
    s = smtplib.SMTP('smtp.gmail.com: 587')
    s.starttls()
    
    # credenciais de login para envio de e-mail:
    s.login(msg['From'], password)
    s.send_message(msg)
    
    print('Email enviado')

In [11]:
enviar_email(lista_produtos_pesquisados, data, str_tabela_google_shop, str_tabela_buscape)

Email enviado
